In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import my_txtutils
import re
import nltk

In [13]:
mypath='./new_txts/'
txts='./txts/'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(txts) if isfile(join(txts, f))]
print(onlyfiles)

['acts_new.txt', 'gal_eph_new.txt', 'heb_new.txt', 'jam_jud_new.txt', 'john_new.txt', 'jud_rev_new.txt', 'luke_8_john_new.txt', 'mark01_new.txt', 'matt02_new.txt', 'matt_new.txt', 'phil_col_new.txt', 'thes_tim_new.txt', 'tit_phl_new.txt']


In [261]:
#large files
def large_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open('path/to/output/file', 'w') as outfile:
        for fname in onlyfiles:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)

In [22]:
#small files
def small_f():
    #filenames = ['file1.txt', 'file2.txt', ...]
    with open(mypath+'full.txt', 'w') as outfile:
        for fname in onlyfiles:
            print('writing file',fname)
            with open(txts+fname) as infile:
                outfile.write(infile.read())
    return open(mypath+"full.txt").read()

In [24]:
txt=small_f()
words=re.sub(' +', ' ',txt.replace('\n',' ')).split(' ')
len(words)

writing file acts_new.txt
writing file gal_eph_new.txt
writing file heb_new.txt
writing file jam_jud_new.txt
writing file john_new.txt
writing file jud_rev_new.txt
writing file luke_8_john_new.txt
writing file mark01_new.txt
writing file matt02_new.txt
writing file matt_new.txt
writing file phil_col_new.txt
writing file thes_tim_new.txt
writing file tit_phl_new.txt


649190

In [25]:
len(txt)

6507438

In [3]:
# these must match what was saved !
ALPHASIZE = my_txtutils.ALPHASIZE
NLAYERS = 3
INTERNALSIZE = 512

ok02 ="checkpoints/rnn_train_1560839489-30000000" # okrika 2019-06-18
ok03 ="checkpoints/rnn_train_1563601394-60000000" # okrika 2019-07-20

author=ok03

ncnt = 0
with tf.Session() as sess:
    # new_saver = tf.train.import_meta_graph('checkpoints/rnn_train_1512567262-0.meta')
    new_saver = tf.train.import_meta_graph(author+'.meta')
    new_saver.restore(sess, author)
    x = my_txtutils.convert_from_alphabet(ord("L"))
    x = np.array([[x]])  # shape [BATCHSIZE, SEQLEN] with BATCHSIZE=1 and SEQLEN=1

    # initial values
    y = x
    h = np.zeros([1, INTERNALSIZE * NLAYERS], dtype=np.float32)  # [ BATCHSIZE, INTERNALSIZE * NLAYERS]
    #for i in range(10):
    yo, h = sess.run(['Yo:0', 'H:0'], feed_dict={'X:0': y, 'pkeep:0': 1., 'Hin:0': h, 'batchsize:0': 1})
    print(tf.shape(yo))
    print(yo)

# Softmax layer implementation:
# Flatten the first two dimension of the output [ BATCHSIZE, SEQLEN, ALPHASIZE ] => [ BATCHSIZE x SEQLEN, ALPHASIZE ]
# then apply softmax readout layer. This way, the weights and biases are shared across unrolled time steps.
# From the readout point of view, a value coming from a sequence time step or a minibatch item is the same thing.
'''
Yflat = tf.reshape(Yr, [-1, INTERNALSIZE])    # [ BATCHSIZE x SEQLEN, INTERNALSIZE ]
Ylogits = layers.linear(Yflat, ALPHASIZE)     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Yflat_ = tf.reshape(Yo_, [-1, ALPHASIZE])     # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Yflat_)  # [ BATCHSIZE x SEQLEN ]
loss = tf.reshape(loss, [batchsize, -1])      # [ BATCHSIZE, SEQLEN ]
Yo = tf.nn.softmax(Ylogits, name='Yo')        # [ BATCHSIZE x SEQLEN, ALPHASIZE ]
Y = tf.argmax(Yo, 1)                          # [ BATCHSIZE x SEQLEN ]
Y = tf.reshape(Y, [batchsize, -1], name="Y")  # [ BATCHSIZE, SEQLEN ]
train_step = tf.train.AdamOptimizer(lr).minimize(loss)
'''


W0721 11:44:55.502967 4466668992 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Tensor("Shape:0", shape=(2,), dtype=int32)
[[0.02731734 0.00911933 0.02987786 0.00783274 0.00781383 0.00791675
  0.00792072 0.00797469 0.00883834 0.00889832 0.00916244 0.00986102
  0.00820671 0.00829697 0.01128333 0.01053004 0.01098963 0.00840346
  0.00767191 0.00813099 0.00805331 0.00803352 0.00793198 0.00814621
  0.00811185 0.00792489 0.00794537 0.00821406 0.00825582 0.00800292
  0.00787085 0.00767554 0.0081084  0.0079169  0.00810332 0.00786511
  0.00813212 0.00800238 0.00778155 0.0079629  0.00791606 0.00817212
  0.00830134 0.0081915  0.0080427  0.00812944 0.00783252 0.00802331
  0.00802122 0.00825228 0.0078577  0.00826596 0.00796773 0.00799554
  0.00809396 0.00801248 0.00795189 0.00826468 0.00790384 0.00775188
  0.00811648 0.00898477 0.00874621 0.0090804  0.00801829 0.00848367
  0.00828231 0.0197167  0.01689533 0.01080387 0.01321569 0.0178067
  0.01208614 0.01350665 0.0111219  0.02097857 0.01208391 0.01571456
  0.01318455 0.01616486 0.01680871 0.01884054 0.01438852 0.00845549
  0.01

In [7]:
print(my_txtutils.convert_from_alphabet(ord("L")))
my_txtutils.encode_text("L")

46


[46]

In [36]:
txt[0:100]

'ÿþ \x00o\x00k\x00u\x00n\x00w\x00e\x00n\x00g\x00i\x00a\x00p\x00u\x00 \x00m\x00i\x00e\x00y\x00e\x00 \x00m\x00a\x00 \x00 \x00d\x00i\x00e\x00p\x00a\x00k\x00u\x00m\x00a\x00y\x00e\x00 \x00 \x00o\x00k\x00u\x00n\x00w\x00e\x00n\x00g\x00i\x00a\x00p\x00u\x00'